In [7]:
import os
import sys
import torch
from torch.utils.data import DataLoader
import math
import torch.nn as nn

sys.path.append(os.path.dirname(os.getcwd()))

from data_loaders import load_data

In [33]:
root_path = os.path.dirname(os.getcwd())
data_path = os.path.join(root_path, "data")

# inputs and outputs parameters
model_name = "transformer_encoder_mlp_decoder"
device = "cpu"
dataset_name = "springs"
num_atoms = 5
temperature = 0.1
length = 1000
num_samples = 1000
n_lags = None

# training parameters
n_epochs = 500
learning_rate = 5e-4
batch_size = 256
batch_first = True
input_size = num_atoms
embedd_hidden_size = 100

suffix = "_" + dataset_name + str(num_atoms)

if (temperature is not None):
    suffix += "_inter" + str(temperature)

if (length is not None):
    suffix += "_l" + str(length)

if num_samples != 50000:
    suffix += "_s" + str(num_samples)

if n_lags is not None:
    suffix += "_lag" + str(n_lags)

## load data
train_dataset = load_data(root_dir=os.path.join(data_path, dataset_name),
                            suffix=suffix,
                            num_atoms=num_atoms)

# NOTE: Random sampling occurs in the "num_sample" dimension instead of "num_obs"
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# choose batch and get src, trg's
i, (features, gt_edges)  = next(enumerate(train_loader))

# reshape features from graph-type data to timeseries-type data
graph_features = features.reshape(features.shape[0], features.shape[2] * features.shape[3], features.shape[1])

src = graph_features

In [34]:
print("[batch_size, num_objects, num_timesteps, num_feature_per_objects]")
features.shape

[batch_size, num_objects, num_timesteps, num_feature_per_objects]


torch.Size([256, 5, 9, 4])

In [35]:
print("[batch_size, num_timesteps * num_feature_per_objects, num_objects]")
src.shape

[batch_size, num_timesteps * num_feature_per_objects, num_objects]


torch.Size([256, 36, 5])

In [36]:
# permute if batch_first == False
if batch_first == False:
    shape_before = src.shape
    src = src.permute(1, 0, 2)

# define ffnn for the embedding dimension
encoder_input_layer = nn.Linear(
    in_features=input_size, 
    out_features=embedd_hidden_size 
    )

# generate embedding dimension from src
x = encoder_input_layer(src)

In [39]:
print("[batch_size, num_timesteps * num_feature_per_objects, num_embeddings]")
x.shape

[batch_size, num_timesteps * num_feature_per_objects, num_embeddings]


torch.Size([256, 36, 100])

In [40]:
d_model = embedd_hidden_size
max_seq_len = 500
dropout = 0.1

position = torch.arange(max_seq_len).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))

nndropout = nn.Dropout(p=dropout)  

if batch_first:
    pe = torch.zeros(1, max_seq_len, d_model)
    pe[0, :, 0::2] = torch.sin(position * div_term)
    pe[0, :, 1::2] = torch.cos(position * div_term)
else:
    pe = torch.zeros(max_seq_len, 1, d_model)
    pe[:, 0, 0::2] = torch.sin(position * div_term)
    pe[:, 0, 1::2] = torch.cos(position * div_term)

if batch_first:
    new_x = x + pe[:,:x.size(1)]
else:
    new_x = x + pe[:x.size(0)]

out = nndropout(new_x)

In [56]:
print("{ x:", x.shape, "+ pe:", pe[:,:x.size(1)].shape, "} = new_x:", new_x.shape)

{ x: torch.Size([256, 36, 100]) + pe: torch.Size([1, 36, 100]) } = new_x: torch.Size([256, 36, 100])
